In [10]:
import cx_Oracle
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import re
from matplotlib import rc
rc("font", family = "Malgun Gothic")
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, r2_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve
from sklearn.metrics import mean_squared_error, mean_squared_log_error 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error



import warnings
warnings.filterwarnings("ignore")

In [2]:
traindata = pd.read_pickle("dataSet/regtrain.pkl")
testdata = pd.read_pickle("dataSet/regtest.pkl")
traindata = traindata[traindata["선호제휴점포"].isin(traindata["선호제휴점포"].value_counts().index[traindata["선호제휴점포"].value_counts() > 30])]
testdata = testdata[testdata["선호제휴점포"].isin(testdata["선호제휴점포"].value_counts().index[testdata["선호제휴점포"].value_counts() > 30])]

In [3]:
traindata

,성별,연령대,거주지역,선호제휴점포,14_2변화율,14_3변화율,14_4변화율,15_1변화율,15_2변화율,R,F,M,Label
0,F,노년층,부산,A29,124.53,78.47,101.10,93.27,105.35,5.0,808.0,25573754.0,66.22
1,F,노년층,서울,A2,66.70,148.41,100.79,122.77,183.79,2.0,762.0,69352903.0,181.64
2,F,노년층,서울,B7,99.38,118.92,96.44,62.93,159.89,1.0,1531.0,12013572.0,124.31
4,F,노년층,서울,A20,55.13,163.19,60.89,136.93,169.26,1.0,587.0,48634778.0,316.62
5,F,노년층,서울,A2,63.31,376.14,26.44,145.20,124.81,2.0,935.0,98499662.0,321.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19368,F,장년층,서울,A1,310.59,121.13,106.16,66.20,85.28,5.0,687.0,29038986.0,25.91
19374,F,청년층,서울,A1,35.55,129.56,100.34,99.04,51.76,2.0,454.0,5881830.0,101.52
19377,M,중년층,대구,B98,156.81,99.35,104.30,89.94,153.47,4.0,1612.0,10957528.0,137.27
19378,F,청년층,서울,B27,110.49,147.17,75.67,1427.06,99.72,5.0,518.0,26054581.0,1165.86


In [3]:
traindata["14_2변화율"] = np.log1p(traindata["14_2변화율"])
traindata["14_3변화율"] = np.log1p(traindata["14_3변화율"])
traindata["14_4변화율"] = np.log1p(traindata["14_4변화율"])
traindata["15_1변화율"] = np.log1p(traindata["15_1변화율"])
traindata["15_2변화율"] = np.log1p(traindata["15_2변화율"])
traindata["F"] = np.log1p(traindata["F"])
traindata["M"] = np.log1p(traindata["M"])
traindata["Label"] = np.log1p(traindata["Label"])

In [4]:
testdata["14_3변화율"] = np.log1p(testdata["14_3변화율"])
testdata["14_4변화율"] = np.log1p(testdata["14_4변화율"])
testdata["15_1변화율"] = np.log1p(testdata["15_1변화율"])
testdata["15_2변화율"] = np.log1p(testdata["15_2변화율"])
testdata["15_3변화율"] = np.log1p(testdata["15_3변화율"])
testdata["F"] = np.log1p(testdata["F"])
testdata["M"] = np.log1p(testdata["M"])
testdata["Label"] = np.log1p(testdata["Label"])

In [7]:
train_ohe = pd.get_dummies(traindata)
test_ohe = pd.get_dummies(testdata)
y_train = train_ohe["Label"]
y_test = test_ohe["Label"]

X_train = train_ohe.drop(["Label"], axis = 1)
X_test = test_ohe.drop(["Label"], axis = 1)

In [9]:
ridge = Ridge()

ridge.fit(X_train, y_train)
pred = ridge.predict(X_test)

In [16]:
pred = np.maximum(pred, 0)

In [17]:
np.sqrt(mean_squared_log_error(y_test, pred))

0.11553691937173877